In [1]:
!pip install faker

In [2]:
import pandas
import faker
import sqlalchemy
import pyodbc # Este é o mais provável de precisar instalar
print("Todas as bibliotecas importadas com sucesso!")

Todas as bibliotecas importadas com sucesso!


In [3]:
# populate_tms_source.py

import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta
from sqlalchemy import create_engine, text
import urllib # Importe urllib para codificar a string de conexão

# --- Configurações do Banco de Dados ---
# AJUSTADAS PARA SUAS INFORMAÇÕES E AUTENTICAÇÃO DO WINDOWS
DB_SERVER = 'BRUNO\\SQLEXPRESS'  # Importante: Use BARRA DUPLA para escapar a barra invertida
DB_DATABASE = 'LoggiConnect'
DB_USERNAME = 'BRUNO\\Bruno Queles' # Seu usuário do Windows (para referência/documentação)
DB_PASSWORD = '' # Senha vazia para Autenticação do Windows
DB_DRIVER = '{ODBC Driver 17 for SQL Server}' # Mantenha a versão do seu driver ODBC
                                              # Se tiver uma versão diferente (ex: 18), ajuste aqui

# --- Inicializa o Faker para geração de dados aleatórios ---
fake = Faker('pt_BR') # 'pt_BR' para dados brasileiros

# --- Dados de Referência (que você já tem no seu DB) ---
CLIENT_IDS = list(range(1, 11)) # Assumindo 10 clientes de ID 1 a 10
TRANSPORTADORA_IDS = list(range(1, 6)) # Assumindo 5 transportadoras de ID 1 a 5
UFS_SUDESTE = ['SP', 'RJ', 'MG', 'ES']

# IDs de Tipos de Ocorrência (Status) - Mapeamento conforme sua tabela Logistica.TiposOcorrencia
STATUS_IDS = {
    'Coleta Realizada': 10,
    'Em Trânsito': 11,
    'Objeto em Rota de Entrega': 12,
    'Entrega Realizada': 13,
    'Atraso na Entrega': 1,
    'Extravio de Encomenda': 2,
    'Avaria no Produto': 3,
    'Destinatário Ausente': 4,
    'Endereço Incorreto': 5,
    'Recusa da Mercadoria': 6,
    'Coleta Não Realizada': 7,
    'Roubo / Furto': 8,
    'Restrição de Entrega': 9,
    'Objeto Devolvido': 14,
    'Problema Fiscal': 15,
}

# Lista de Cidades por UF (simplificada para o exemplo)
CIDADES_POR_UF = {
    'SP': ['São Paulo', 'Campinas', 'Guarulhos', 'Ribeirão Preto', 'Santos'],
    'RJ': ['Rio de Janeiro', 'Niterói', 'Duque de Caxias', 'Nova Iguaçu', 'Campos dos Goytacazes'],
    'MG': ['Belo Horizonte', 'Uberlândia', 'Contagem', 'Juiz de Fora', 'Montes Claros'],
    'ES': ['Vitória', 'Vila Velha', 'Serra', 'Cariacica', 'Linhares']
}

# --- Configurações de Geração de Dados em Massa para a TMS_Fonte ---
NUM_TOTAL_OPERATIONS_FOR_TMS_YEAR = 100000 # Objetivo: Gerar 100.000 operações ANUAIS para a TMS_Fonte
SIMULATION_YEAR = 2025 # O ANO DE SIMULAÇÃO PARA A FONTE É 2025

# Feriados e seus fatores de aumento (ex: 2.0 significa dobro de operações)
HOLIDAY_BOOSTS = {
    "2025-04-18": 1.5,  # Sexta-feira Santa (Páscoa) em 2025
    "2025-05-11": 1.3,  # Dia das Mães em 2025
    "2025-06-12": 1.2,  # Dia dos Namorados
    "2025-09-07": 1.1,  # Independência do Brasil
    "2025-10-12": 1.3,  # Dia das Crianças
    "2025-11-28": 2.5,  # Black Friday (última sexta-feira de novembro em 2025)
    "2025-12-25": 2.0,  # Natal
}
# Fatores de volume por mês (base 1.0)
MONTHLY_BOOSTS = {
    1: 0.9,  # Janeiro (pós-festas)
    2: 0.9,  # Fevereiro
    3: 1.0,  # Março
    4: 1.0,  # Abril
    5: 1.1,  # Maio (Dia das Mães)
    6: 1.0,  # Junho (Dia dos Namorados)
    7: 1.0,  # Julho
    8: 1.0,  # Agosto
    9: 1.0,  # Setembro
    10: 1.1, # Outubro (Dia das Crianças)
    11: 1.5, # Novembro (Black Friday)
    12: 1.8, # Dezembro (Natal)
}

# --- Função para Gerar um Único Registro de Operação ---
def generate_single_operation_data(current_date_for_op, op_id_counter):
    """
    Gera um único registro de dados para a tabela Logistica.Operacoes_TMS_Fonte.
    """
    data_pedido = fake.date_time_between(start_date=current_date_for_op, end_date=current_date_for_op + timedelta(days=1), tzinfo=None)

    id_cliente = random.choice(CLIENT_IDS)
    id_transportadora = random.choice(TRANSPORTADORA_IDS)

    origem_uf = random.choice(UFS_SUDESTE)
    origem_cidade = random.choice(CIDADES_POR_UF[origem_uf])

    destino_uf = random.choice(UFS_SUDESTE)
    destino_cidade = random.choice(CIDADES_POR_UF[destino_uf])

    # Garante que origem e destino não sejam idênticos na maioria dos casos
    while origem_uf == destino_uf and origem_cidade == destino_cidade:
        destino_uf = random.choice(UFS_SUDESTE)
        destino_cidade = random.choice(CIDADES_POR_UF[destino_uf])

    peso_kg = round(random.uniform(0.1, 50.0), 2) # Peso entre 0.1kg e 50kg
    volume_m3 = round(random.uniform(0.001, 0.5), 4) # Volume entre 0.001m3 e 0.5m3

    base_frete = 10 + peso_kg * 0.5 + volume_m3 * 10
    valor_frete = round(base_frete * random.uniform(0.8, 1.5), 2)

    previsao_dias = random.randint(2, 10)
    data_previsao_entrega = (data_pedido + timedelta(days=previsao_dias)).date()

    rand_status = random.random()
    if rand_status < 0.80:
        current_status = STATUS_IDS['Entrega Realizada']
        data_entrega_real = (data_pedido + timedelta(days=random.randint(1, previsao_dias))).date()
    elif rand_status < 0.95:
        current_status = STATUS_IDS['Em Trânsito']
        data_entrega_real = None
    else:
        problem_status_options = [
            STATUS_IDS['Atraso na Entrega'],
            STATUS_IDS['Destinatário Ausente'],
            STATUS_IDS['Problema Fiscal'],
        ]
        current_status = random.choice(problem_status_options)
        data_entrega_real = None

    codigo_rastreio = f'LC{op_id_counter:08d}{random.randint(0,999):03d}' # Código de rastreio único

    return {
        'id_cliente': id_cliente,
        'id_transportadora': id_transportadora,
        'data_pedido': data_pedido,
        'data_previsao_entrega': data_previsao_entrega,
        'data_entrega_real': data_entrega_real,
        'origem_uf': origem_uf,
        'origem_cidade': origem_cidade,
        'destino_uf': destino_uf,
        'destino_cidade': destino_cidade,
        'peso_kg': peso_kg,
        'volume_m3': volume_m3,
        'valor_frete': valor_frete,
        'status_entrega_id': current_status,
        'codigo_rastreio': codigo_rastreio
    }

# --- Função para Obter Conexão com o Banco de Dados ---
def get_db_connection():
    """
    Cria e retorna um objeto de conexão SQLAlchemy para o SQL Server
    usando Autenticação do Windows (Trusted Connection).
    """
    params = urllib.parse.quote_plus(
        f"DRIVER={DB_DRIVER};"
        f"SERVER={DB_SERVER};"
        f"DATABASE={DB_DATABASE};"
        f"Trusted_Connection=yes;"
    )
    conn_str = f"mssql+pyodbc:///?odbc_connect={params}"
    engine = create_engine(conn_str)
    return engine

# --- Bloco Principal: População da Logistica.Operacoes_TMS_Fonte ---
if __name__ == "__main__":
    engine = get_db_connection()

    # --- Limpa dados existentes da tabela Logistica.Operacoes_TMS_Fonte ---
    # Isso garante que a TMS_Fonte sempre contenha apenas o ano completo de 2025
    print("\n--- Limpando dados existentes da tabela Logistica.Operacoes_TMS_Fonte ---")
    try:
        with engine.connect() as connection:
            connection.execute(text("TRUNCATE TABLE Logistica.Operacoes_TMS_Fonte;"))
            print("Tabela Logistica.Operacoes_TMS_Fonte truncada com sucesso.")
            connection.commit()
    except Exception as e:
        print(f"Erro ao truncar tabela Logistica.Operacoes_TMS_Fonte: {e}")
        print("Verifique permissões ou se a tabela está em uso. Saindo do script.")
        exit() # Sai do script se não conseguir truncar, para evitar duplicação indesejada

    # --- Obter o ID da última operação existente (para garantir a numeração se o IDENTITY resetar) ---
    # Para a TMS_Fonte, como estamos truncando, o ID sempre começará de 1, mas é boa prática para outros cenários.
    last_op_id = 0 # Sempre começará de 0, pois a tabela será truncada
    current_op_id_counter = last_op_id + 1 # Inicia o contador do próximo ID disponível

    # Definindo o período completo para a TMS_Fonte (todo o ano de 2025)
    TMS_SOURCE_START_DATE = datetime(SIMULATION_YEAR, 1, 1)
    TMS_SOURCE_END_DATE = datetime(SIMULATION_YEAR, 12, 31)

    # Calculamos o número de operações por dia baseadas no total ANUAL e no período de geração
    total_days_in_year = (TMS_SOURCE_END_DATE - TMS_SOURCE_START_DATE).days + 1 # +1 para incluir o último dia
    base_daily_operations_raw = NUM_TOTAL_OPERATIONS_FOR_TMS_YEAR / total_days_in_year

    print(f"\nIniciando a geração de ~{NUM_TOTAL_OPERATIONS_FOR_TMS_YEAR} operações para o ano de {SIMULATION_YEAR} (para a TMS_Fonte)...")
    all_operations_data_for_tms = []
    
    current_date_loop = TMS_SOURCE_START_DATE
    while current_date_loop <= TMS_SOURCE_END_DATE:
        day_str = current_date_loop.strftime("%Y-%m-%d")
        
        daily_factor = HOLIDAY_BOOSTS.get(day_str, 1.0)
        monthly_factor = MONTHLY_BOOSTS.get(current_date_loop.month, 1.0)

        num_ops_today = int(base_daily_operations_raw * daily_factor * monthly_factor)
        num_ops_today = int(num_ops_today * random.uniform(0.8, 1.2)) # Variação diária

        if num_ops_today < 1:
            num_ops_today = 1
            
        for _ in range(num_ops_today):
            all_operations_data_for_tms.append(generate_single_operation_data(current_date_loop, current_op_id_counter))
            current_op_id_counter += 1

        current_date_loop += timedelta(days=1)

    df_tms_operations = pd.DataFrame(all_operations_data_for_tms)
    print(f"Total de {len(df_tms_operations)} operações geradas em memória para a Logistica.Operacoes_TMS_Fonte.")
    # print(df_tms_operations.head())
    # print(df_tms_operations.tail())


    # --- Inserindo dados na tabela Logistica.Operacoes_TMS_Fonte ---
    print(f"\n--- Inserindo {len(df_tms_operations)} dados na tabela Logistica.Operacoes_TMS_Fonte. Isso pode levar um bom tempo... ---")
    try:
        df_tms_operations.to_sql(
            name='Operacoes_TMS_Fonte',
            con=engine,
            schema='Logistica',
            if_exists='append', # Adiciona (após o truncamento)
            index=False,
            chunksize=1000
        )
        print(f"Todos os {len(df_tms_operations)} dados foram inseridos com sucesso na tabela Logistica.Operacoes_TMS_Fonte!")
    except Exception as e:
        print(f"Erro ao inserir dados na Logistica.Operacoes_TMS_Fonte: {e}")
        print("Verifique se as colunas do DataFrame correspondem às da tabela Logistica.Operacoes_TMS_Fonte no SQL Server.")

    # --- Testando conexão com o SQL Server (mantido) ---
    print("\n--- Testando conexão com o SQL Server ---")
    try:
        with engine.connect() as connection:
            result = connection.execute(text("SELECT 1 AS ConnectionTest")).scalar()
            if result == 1:
                print("Conexão com o SQL Server estabelecida com sucesso!")
            else:
                print("Erro inesperado ao testar a conexão com o SQL Server.")
    except Exception as e:
        print(f"Erro ao conectar ao SQL Server: {e}")
        print("Verifique seu servidor, nome da instância, nome do banco de dados e driver ODBC.")


--- Limpando dados existentes da tabela Logistica.Operacoes_TMS_Fonte ---
Tabela Logistica.Operacoes_TMS_Fonte truncada com sucesso.
Erro ao truncar tabela Logistica.Operacoes_TMS_Fonte: 'Connection' object has no attribute 'commit'
Verifique permissões ou se a tabela está em uso. Saindo do script.

Iniciando a geração de ~100000 operações para o ano de 2025 (para a TMS_Fonte)...
Total de 111361 operações geradas em memória para a Logistica.Operacoes_TMS_Fonte.

--- Inserindo 111361 dados na tabela Logistica.Operacoes_TMS_Fonte. Isso pode levar um bom tempo... ---
Todos os 111361 dados foram inseridos com sucesso na tabela Logistica.Operacoes_TMS_Fonte!

--- Testando conexão com o SQL Server ---
Conexão com o SQL Server estabelecida com sucesso!
